MakerSuite embeddings, using embeddings to convert the text to vectorized and on that classify it using SVM.

In [19]:
import google.generativeai as palm

import re
import tqdm
import tensorflow as tf
import keras
import numpy as np
import sklearn as sk
import pandas as pd

from keras import layers
from sklearn.model_selection import train_test_split
from google.api_core import retry
from tqdm.auto import tqdm
import sys
import platform

Making sure that the kernals are correctly in use and that all resources are available

In [20]:
print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Setting the api key and the model

In [21]:
palm.configure(api_key='<>')
models = [m for m in palm.list_models() if 'embedText' in m.supported_generation_methods]

model = models[0]

Preparing the data and create a train set

In [31]:
# Import the data, and spit the data
data = pd.read_csv("StaterData.csv")

issue_to_int = {
    "Trouble during payment process": 0,
    "Struggling to pay mortgage": 1,
    "Loan servicing, payments, escrow account": 2,
    "Applying for a mortgage or refinancing an existing mortgage": 3,
    "Loan modification,collection,foreclosure": 4,
    "Closing on a mortgage": 5,
    "Application, originator, mortgage broker": 6,
    "Credit decision / Underwriting": 7,
    "Incorrect information on your report": 8,
    "Settlement process and costs": 9,
    "Problem with a credit reporting company's investigation into an existing problem": 10,
    "Improper use of your report": 11,
    "Credit monitoring or identity theft protection services": 12,
}

# Create a new column called `Issue_Int`
data["Issue_Int"] = data["Issue"].map(issue_to_int)

df_train, df_test = train_test_split(data, test_size=0.3, random_state=2)


Calling the API from Palm and convert the text to vectors, PaLM api limit is currently 4 items per second

In [32]:
# Python progress bar 
tqdm.pandas()

def make_embed_text_fn(model):
  @retry.Retry(timeout=300.0)
  def embed_fn(text: str) -> list[float]:
    # Using the palm model generate the embeddings for 
    return palm.generate_embeddings(model=model, text=text)['embedding']
  return embed_fn

# Creates the embedding of the send dataframe
def create_embeddings(model, data):
  # Adds the column embeddings with the corrosponding embedding
  data['Embeddings'] = data['Consumer complaint narrative'].progress_apply(make_embed_text_fn(model))
  return data

# Creates the embedding for the train and the testdata
data = create_embeddings(model, data)


  0%|          | 0/105103 [00:00<?, ?it/s]

Build the Keras model, used for the classification

In [ ]:
def build_classification_model(input_size: int, num_classes: int) -> keras.Model:
  inputs = x = keras.Input(input_size)
  x = layers.Dense(input_size, activation='relu')(x)
  x = layers.Dense(num_classes, activation='sigmoid')(x)
  return keras.Model(inputs=[inputs], outputs=x)

# Derive the embedding size from the first training element.
embedding_size = len(df_train['Embeddings'].iloc[0])

# Give your model a different name, as you have already used the variable name 'model'
classifier = build_classification_model(embedding_size, len(df_train['Consumer complaint narrative'].unique()))
classifier.summary()

classifier.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   optimizer = keras.optimizers.Adam(learning_rate=0.001),
                   metrics=['accuracy'])


Train the model on the dataset

In [ ]:
NUM_EPOCHS = 20
BATCH_SIZE = 8

# Split the x and y components of the train and validation subsets.
y_train = df_train['Issue_Int']
x_train = np.stack(df_train['Embeddings'])
y_val = df_test['Issue_Int']
x_val = np.stack(df_test['Embeddings'])

# Train the model for the desired number of epochs.
callback = keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

history = classifier.fit(x=x_train,
                         y=y_train,
                         validation_data=(x_val, y_val),
                         callbacks=[callback],
                         batch_size=BATCH_SIZE,
                         epochs=NUM_EPOCHS,) 

Evaluate how accurate the model is

In [ ]:
classifier.evaluate(x=x_val, y=y_val, return_dict=True)

Create 2 plots that show the accuracy over the amounts of epochs and loses

In [ ]:
def plot_history(history):
  fig, (ax1, ax2) = plt.subplots(1,2)
  fig.set_size_inches(20, 8)

  # Plot loss
  ax1.set_title('Loss')
  ax1.plot(history.history['loss'], label = 'train')
  ax1.plot(history.history['val_loss'], label = 'test')
  ax1.set_ylabel('Loss')

  ax1.set_xlabel('Epoch')
  ax1.legend(['Train', 'Validation'])

  # Plot accuracy
  ax2.set_title('Accuracy')
  ax2.plot(history.history['accuracy'],  label = 'train')
  ax2.plot(history.history['val_accuracy'], label = 'test')
  ax2.set_ylabel('Accuracy')
  ax2.set_xlabel('Epoch')
  ax2.legend(['Train', 'Validation'])

  plt.show()

plot_history(history)

The model shows great promise, without cleaning the text the model almost gives the same accuracy as using TF-IDF with cleaning